# Práctica 2: Aprendizaje automático

__Fecha de entrega: 16 de mayo de 2021__

El objetivo de esta práctica es aplicar los distintos algoritmos de aprendizaje automático disponibles en la scikit-learn [sklearn](https://scikit-learn.org/stable/) sobre varios conjuntos de datos y aprender a interpretar los resultados obtenidos. La práctica consta de 3 notebooks que se entregarán simultáneamente en la tarea de entrega habilitada en el Campus  Virtual.

Lo más importante en esta práctica no es el código Python, sino el análisis de los datos y modelos que construyas y las explicaciones razonadas de cada una de las decisiones que tomes. __No se valorarán trozos de código o gráficas sin ningún tipo de contexto o explicación__.

Finalmente, recuerda establecer el parámetro `random_state` en todas las funciones que tomen decisiones aleatorias para que los resultados sean reproducibles (los resultados no varíen entre ejecuciones).

# Apartado 1: Clustering

__Número de grupo: XX__

__Nombres de los estudiantes: XXX y XXX__

## 1) Carga del conjunto de datos

Crea un dataframe a partir del fichero `countries_of_the_world.csv` que se proporciona junto con la práctica. Usa como índice el nombre de los países. 

Vamos a eliminar la columna `Region` por ser categórica y todas las filas en las que faltan valores usando la operación `dropna`.

Muestra el dataframe resultante y explica cuántos países y variables contiene.

In [1]:
import pandas as pd
df = pd.read_csv('countries_of_the_world.csv')
df.set_index("Nombre")
df.drop(['Region'], axis=1)/*axis=0 fila; axis = 1 columna*/
df.dropna()
print(df)

SyntaxError: invalid syntax (<ipython-input-1-7503bc3fc9a2>, line 4)

## 2) Análisis de los datos

En este notebook vamos a trabajar con un subconjunto de las variables. Crea un nuevo dataframe que sólo contenga las variables `GDP ($ per capita)`, `Literacy (%)`, `Phones (per 1000)`, `Agriculture`, `Industry` y `Service`. ¿Qué crees que representan cada una de esas variables?

Analiza razonadamente las distribuciones de cada una de las variables (medias, desviaciones típicas, rangos, ...) y las principales relaciones entre pares de variables (diagrama de dispersión, coeficientes de correlación, ...).

## 3) Preprocesar los datos

Teniendo en cuenta que vamos a utilizar el algoritmo k-Means para encontrar grupos de países similares, explica razonadamente si es necesario o no cambiar la escalas de los datos y si a priori es mejor reescalarlos (MinMaxScaler) o estandarizarlos (StandarScaler).

Si decides preprocesarlos, accede al array interno del dataframe y crea un nuevo array con los datos escalados.

In [ ]:
#tenemos que agrupar en funcion de X, y despues lo utilizamos para los clusters
#ya que en nuestro archivo no manejaremos numeros negativos hemos decidido normalizar(MinMaxScaler)
datos_escalados = preprocessing.MinMaxScaler(feature_range=(0, 1))
datos_escalados = datos_escalados.fit_transform(df)
print("Datos escalados:\n", datos_escalados)

# buscado en internet
Es mas util normalizar, ya que no manejaremos valores valores negativos, y tenemos un rango de valores muy amplio que nos vendra bien escalar.

MinMaxScaler(normalizar segun el profesor) : Es posible que nuestro vector característico incluya valores extremadamente altos y/o extremadamente bajos,
por eso es importante escalar esos valores para que nuestro algoritmo opere con valores cercanos, y así evitamos características con valores desorbitantes.

StandardScaler: Transforma los datos de tal manera que tiene una media de 0 y una desviación estándar de 1.
En resumen, estandariza los datos . La estandarización es útil para datos que tienen valores negativos.
Se organiza los datos en una distribución normal estándar . Es más útil en clasificación que en regresión.

Normalizer: Exprime los datos entre 0 y 1. Realiza la normalización .
Debido a la disminución del rango y la magnitud, los gradientes en el proceso de entrenamiento no explotan y no obtienes valores más altos de pérdida.
Es más útil en regresión que en clasificación.

## 4) Encontrar el número óptimo de clusters

Decide razonadamente el número óptimo de clusters en el rango 2..10. Ten en cuenta que para interpretar los datos no nos interesa tampoco tener un número excesivo de clusters. Para hacerlo calcula y pinta el diagrama del codo, el índice davies_boulding y el coeficiente silhouette en función del número de clusters.

In [ ]:
from sklearn.metrics import davies_bouldin_score, silhouette_score

K_MAX = 10
score = np.zeros(K_MAX-2)
davies_boulding = np.zeros(K_MAX-2)
silhouette = np.zeros(K_MAX-2)
for k in range(2, K_MAX): 
    km = KMeans(init='random', n_clusters=k, random_state=RANDOM_STATE)
    km.fit(X)
    plot_clusters(X, km.labels_, km.cluster_centers_)
    
    score[k-2] = -1 * km.score(X)
    davies_boulding[k-2] = davies_bouldin_score(X, km.labels_)
    silhouette[k-2] = silhouette_score(X, km.labels_)
    
#mostraremos los tres diagramas para comparar y comprobar cual es el numero optimo de clusters
# diagrama del codo
plt.plot(range(2, K_MAX), score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
#indice davies boulding
plt.plot(range(2, K_MAX), davies_boulding)
plt.xlabel('Number of Clusters')
plt.ylabel('Davies Boulding value')
plt.title('Valor de Davies Boulding para diferentes k')
plt.show()

In [ ]:
# silhouette
plt.plot(range(2, K_MAX), silhouette)
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette value')
plt.title('Valor de Silhouette para diferentes k')
plt.show()

## 5) Descripción de los clusters

Describe los clusters que has obtenido en el apartado anterior y trata identificar el grupo de países que contienen. Si te han salido más de 3 elige 3 de ellos que sean bastante diferentes entre sí. 

Para hacerlo estudia sus descriptores estadísticos y pinta el diagrama de dispersión en función de cada par de variables usando colores diferentes para cada cluster. ¿Qué clusters se separan mejor y en función de qué variables? ¿y cuáles se confunden más?

__Cuidado__: para poder interpretar correctamente los datos necesitas que estén en su escala original. Si decidiste escalar los datos, deberás ejecutar kMeans con los datos escalados pero asignar las etiquetas de clusters al conjunto de datos inicial. En este caso es muy sencillo porque el algoritmo no cambia el orden de los datos así que puedes crear directamente una nueva columna en el dataframe original con esas etiquetas. Puede que aparezca un SettingWithCopyWarning por asignar una nueva columna en un dataframe que es una vista de otro dataframe. Puedes ignorar este aviso o puedes hacer una copia del dataframe con `copy` para que no comparta memoria con el otro.